In [28]:
pip install pandas numpy ipython glob2 tqdm

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for glob2: filename=glob2-0.7-py2.py3-none-any.whl size=9312 sha256=f9e88c54a90adfd15c0f46dff16076d463ca22cd83186a1c905ee81cbcaf4494
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\c0\dd\ee\ba6164807de7570971e8f160dbe6a4178ff4e5922f48c093be
Successfully built glob2
Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import glob
from tqdm import tqdm


# **NASA FIRMS - Fire Information for Resource Management System**

FIRMS distributes Near Real-Time (NRT) active fire data within 3 hours of satellite observation from the Moderate Resolution Imaging Spectroradiometer (MODIS) aboard the Aqua and Terra satellites.

# **Acknowledgement & Disclaimer**

We acknowledge the use of data and/or imagery from NASA's FIRMS (https://earthdata.nasa.gov/firms), part of NASA's Earth Observing System Data and Information System (EOSDIS).


*   Do not use for the preservation of life or property. Satellite-derived active fire / thermal anomalies have limited accuracy.
*   Active fire/thermal anomalies may be from fire, hot smoke, agriculture or other sources.
*   Cloud cover may obscure active fire detections.

# **Defining the Target Zone**


**Define the latitude & longitude ranges of your target country, zone, etc. The example is for greece.**

In [12]:
LAT_RANGE = (34, 42)

**Read and check the chunks**

The archive fire/hotspot datasets could be requested at https://firms.modaps.eosdis.nasa.gov/download/ in yearly chunks for each instrument.

*  MODIS Collection 6.1: Temporal Coverage: 11 November 2000 - present




**Importing Wildfire data merge csv file.**

In [18]:
filenames = glob.glob(r"C:\Users\User\Videos\UTS\ꞮꞮꞮ.Semester\iLab 2\Countries\Greece\Wildfire Prediction\Greece_wildfire_prediction\data\Wildfire data\Wild_Fire_Data.csv")
filenames

['C:\\Users\\User\\Videos\\UTS\\ꞮꞮꞮ.Semester\\iLab 2\\Countries\\Greece\\Wildfire Prediction\\Greece_wildfire_prediction\\data\\Wildfire data\\Wild_Fire_Data.csv']

# **Process each chunk**

We removed fire readings with low or less than 50 confidence. We also took only the zones that falls within our target latitude & longitude ranges. For simplicity the coordinates are rounded to two decimal degrees. That is roughly 1.1 km at the Equator. For better spatial resolution the original VIIRS records could be used.

In [19]:
LAT_RANGE = (34, 42)
LON_RANGE = (19, 29)

chunks = []
cols_to_read = ['latitude', 'longitude', 'acq_date', 'satellite', 'instrument', 'confidence']
for f in tqdm(filenames):
    fire = pd.read_csv(f, usecols=cols_to_read, parse_dates=['acq_date'], low_memory=False)
    if fire.satellite.loc[0] in ['Terra', 'Aqua', 'N']:
        try:
            fire = fire[(fire['latitude'] >= LAT_RANGE[0]) & (fire['latitude'] <= LAT_RANGE[1]) &
                        (fire['longitude'] >= LON_RANGE[0]) & (fire['longitude'] <= LON_RANGE[1])]
            fire['latitude'] = fire['latitude'].round(2)
            fire['longitude'] = fire['longitude'].round(2)
            fire['confidence'] = fire['confidence'].replace({'l': 0, 'n': 50, 'h': 100})
            daily_fires = fire.groupby(
                ['latitude', 'longitude', 'acq_date', 'satellite', 'instrument']).confidence.max().reset_index()
            daily_fires = daily_fires[daily_fires['confidence'] >= 50]  # Remove low confidence records
            chunks.append(daily_fires)

            # Now you can print or access daily_fires within the loop
            print(daily_fires.head())  # Or any other operations you want to perform

        except:
            pass
    else:
        pass


100%|██████████| 1/1 [00:00<00:00, 12.95it/s]

   latitude  longitude    acq_date satellite instrument  confidence
0     34.81      24.12  05/08/2018     Terra      MODIS          62
5     34.94      26.13  26/08/2023      Aqua      MODIS          60
6     34.94      26.14  03/08/2016     Terra      MODIS          66
7     34.94      26.14  13/08/2014     Terra      MODIS          66
8     34.94      26.14  25/08/2023      Aqua      MODIS          62


In [20]:
daily_fires.head()

,latitude,longitude,acq_date,satellite,instrument,confidence
0,34.81,24.12,05/08/2018,Terra,MODIS,62
5,34.94,26.13,26/08/2023,Aqua,MODIS,60
6,34.94,26.14,03/08/2016,Terra,MODIS,66
7,34.94,26.14,13/08/2014,Terra,MODIS,66
8,34.94,26.14,25/08/2023,Aqua,MODIS,62


**We are merging the chunks we created into one full dataframe.**

In [21]:
full_dataset = pd.concat(chunks)
full_dataset

,latitude,longitude,acq_date,satellite,instrument,confidence
0,34.81,24.12,05/08/2018,Terra,MODIS,62
5,34.94,26.13,26/08/2023,Aqua,MODIS,60
6,34.94,26.14,03/08/2016,Terra,MODIS,66
7,34.94,26.14,13/08/2014,Terra,MODIS,66
8,34.94,26.14,25/08/2023,Aqua,MODIS,62
...,...,...,...,...,...,...
17628,41.81,26.13,02/11/2018,Terra,MODIS,60
17629,41.81,26.14,05/03/2017,Aqua,MODIS,84
17630,41.82,26.11,12/10/2013,Terra,MODIS,59
17631,41.82,26.13,20/10/2014,Aqua,MODIS,57


In [24]:
# 'full_dataset' is my DataFrame
full_dataset['acq_date'] = pd.to_datetime(full_dataset['acq_date'], format='%d/%m/%Y')

# Create separate 'Month' and 'Year' columns
full_dataset['month'] = full_dataset['acq_date'].dt.month
full_dataset['year'] = full_dataset['acq_date'].dt.year



In [25]:
full_dataset = full_dataset.drop(columns=['acq_date'])

In [26]:
full_dataset

,latitude,longitude,satellite,instrument,confidence,month,year
0,34.81,24.12,Terra,MODIS,62,8,2018
5,34.94,26.13,Aqua,MODIS,60,8,2023
6,34.94,26.14,Terra,MODIS,66,8,2016
7,34.94,26.14,Terra,MODIS,66,8,2014
8,34.94,26.14,Aqua,MODIS,62,8,2023
...,...,...,...,...,...,...,...
17628,41.81,26.13,Terra,MODIS,60,11,2018
17629,41.81,26.14,Aqua,MODIS,84,3,2017
17630,41.82,26.11,Terra,MODIS,59,10,2013
17631,41.82,26.13,Aqua,MODIS,57,10,2014


# **Export**

We are exporting the wildfire data into csv file.

In [27]:
full_dataset.to_csv('wildfire_processed_data.csv', index=False)